In [1]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

AUTHORITIES

In [2]:
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/authority/update')
query_endpoint = 'http://localhost:3030/authority/query'
update_endpoint = 'http://localhost:3030/authority/update'
store.open((query_endpoint, update_endpoint))

Acervo

In [2]:
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/acervo/update')
query_endpoint = 'http://localhost:3030/acervo/query'
update_endpoint = 'http://localhost:3030/acervo/update'
store.open((query_endpoint, update_endpoint))

In [13]:
q = """SELECT ?g  
        {graph ?g {?s ?p ?o}} 
        group by ?g 
        ORDER BY DESC(?g)
        LIMIT 1"""

r = store.query(q)

In [28]:
for i in r:
        uri = i[0].toPython()


In [30]:
bk = uri.split('/')[-1]

In [31]:
bk

'bk231'

DELETE GRAPHS

In [4]:
store.update("DROP DEFAULT")

DELETE NAMED GRAPHS

In [3]:
d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

store.update(d)

POST WORK

In [21]:
work = {
  "ID": "bk-1",
  "content": "Texto",
  "mainTitle": "Conjectures and refutations",
  "subtitle": "the growth of scientific knowledge",
  "primaryContributionAgent": "MACHADÃO de Assis",
  "primaryContributionUri": "https://bibliokeia.com/authorities/names/n79143148",
  "primaryContributionRole": "aut",
  "subjects": [
    {
      "label": "Ciência",
      "uri": "https://bibliokeia.com/authorities/subjects/sh85084414"
    }
  ],
  "language": "Português",
  "languageCode": "por",
  "cdd": "540.2",
  "cutter": "O47p"
}

UPDATE WORK

In [7]:
from pyfuseki import FusekiUpdate, FusekiQuery
from pydantic import BaseModel
from typing import Optional

In [10]:
class Subject(BaseModel):
    label: str
    uri: str
    
class Work_Schema(BaseModel):
    ID: Optional[str]
    content: str
    mainTitle: str
    subtitle: Optional[str]
    variantTitle: Optional[str]
    primaryContributionAgent: str
    primaryContributionUri: str
    primaryContributionRole: str
    subjects: list[Subject]
    language: str
    languageCode: str 
    cdd: str
    cutter: str
    serie: Optional[str]
    serieURI: Optional[str]

In [22]:
request = Work_Schema(**work)

In [27]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

In [38]:
for subject in request.subjects:
    print(subject)

label='Ciência' uri='https://bibliokeia.com/authorities/subjects/sh85084414'


In [42]:
bkID = 'bk-1'
prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""

askAgent = prefix+"ASK { graph work:"+bkID+"""
                    {work:"""+bkID+""" bf:subject <"""+subject.uri+"> . }}"

response = acervoQuery.run_sparql(askAgent)
response = response.convert()
response

{'head': {}, 'boolean': True}

In [35]:
for subject in request.subjects:
    print(subject)

label='Ciência' uri='https://bibliokeia.com/authorities/subjects/sh85084414'


In [30]:

if not response['boolean']:
        up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:contribution ?o .
                        ?o ?p ?agent }
                INSERT {work:"""+bkID+""" bf:contribution ?o .
                ?o rdf:type bf:Contribution .
                ?o rdfs:label '"""+request.primaryContributionAgent+"""' .
                ?o bf:role <http://id.loc.gov/vocabulary/relators/"""+request.primaryContributionRole+"""> .
                ?o bf:agent <"""+request.primaryContributionUri+"""> }
                WHERE {work:"""+bkID+""" bf:contribution ?o .
                        ?o ?p ?agent }"""
        acervoUpdate.run_sparql(up)